In [1]:
import os
os.environ['HF_HOME'] = '/data1/malto/cache'

In [2]:
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from pathlib import Path

BASE_DIR = Path("/data1/malto/shroom/")
BATCH_SIZE = 20
NUM_EPOCHS_SYN = 2
NUM_EPOCHS_GPT = 2
NUM_EPOCHS_TRUE = 2

BASE_DIR = Path("/data1/malto/shroom/")

checkpoint = "microsoft/deberta-xlarge-mnli"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/data1/malto/csavelli/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def preprocess_function(examples): # not batched
    model_inputs = tokenizer(examples['hyp'], examples['tgt'] if examples['ref'] != 'src' else examples['src'], truncation=True, max_length=80)
    return model_inputs

def get_label(examples): # not batched
    return {"label" : 1 if examples['p(Hallucination)'] > 0.5 else 0}

def compute_metrics(eval_pred):
    #print(eval_pred)
    accuracy = evaluate.load("accuracy")
    predictions, labels = eval_pred
    #print(predictions, labels)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "Not Hallucination", 1: "Hallucination"}
label2id = {"Not Hallucination": 0, "Hallucination": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge-mnli and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

syntetic_test_size_split = 0.000000001

ds_mt = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_MT_SOLAR.model-agnostic.json")])
ds_dm = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_DM_SOLAR.model-agnostic.json")])
ds_pg = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_PG_SOLAR.model-agnostic.json")])
ds_val = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-agnostic.json")])
ds_val_aware = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-aware.json")])
ds_gpt = load_dataset("json", data_files=str(BASE_DIR / f"transformed_val_model_gpt.json"))

ds_mt = ds_mt.remove_columns([el for el in ds_mt['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_dm = ds_dm.remove_columns([el for el in ds_dm['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_pg = ds_pg.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_gpt = ds_gpt.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)

ds_syn = concatenate_datasets([ds_mt['train'], ds_dm['train'], ds_pg['train']])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 2816.86it/s]
Generating train split: 501 examples [00:00, 253939.13 examples/s]


In [ ]:
def train_with_dataset(ds, num_epochs):
    ds = ds.map(get_label).map(preprocess_function).remove_columns(['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)']).shuffle()
    training_args = TrainingArguments(
        output_dir=BASE_DIR / "checkpoint" / "sequential",
        learning_rate=1e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        report_to="none",
        save_strategy="no",
        logging_steps=1,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds,
        eval_dataset=ds_val_aware.map(get_label).map(preprocess_function).remove_columns(['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)']),
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer

In [ ]:
ds_syn

In [ ]:
train_with_dataset(ds_syn, NUM_EPOCHS_SYN)

In [ ]:
train_with_dataset(ds_gpt['train'], NUM_EPOCHS_GPT)

In [ ]:
trainer = train_with_dataset(ds_val['train'], NUM_EPOCHS_TRUE)

In [ ]:
def preprocess_function_test(examples): # not batched
    model_inputs = tokenizer(examples['hyp'], examples['tgt'], truncation=True, max_length=80)
    return model_inputs

ds_test = load_dataset("json", data_files=[str(BASE_DIR / f"test.model-agnostic.json")])

In [ ]:
ds_test

In [ ]:
predictions, _, _ = trainer.predict(ds_test['train'].map(preprocess_function_test))

import scipy
import pandas as pd
preds = scipy.special.expit(predictions)
preds = preds[:, 0] / preds.sum(axis=1)

df = pd.DataFrame(preds, columns=["sequential_nogpt_deberta_xlarge"])
df.to_csv(f"/data1/malto/shroom/sequential_nogpt.csv", index=False)

In [ ]:
predictions, _, _ = trainer.predict(ds_val_aware['train'].map(get_label).map(preprocess_function).remove_columns(['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)']))
preds = scipy.special.expit(predictions)
preds = preds[:, 0] / preds.sum(axis=1)

df = pd.DataFrame(preds, columns=["sequential_nogpt_deberta_xlarge"])
df.to_csv(f"/data1/malto/shroom/sequential_nogpt_aware.csv", index=False)